<a href="https://colab.research.google.com/github/NicoCasPer/T_Aprendizaje_Maquina/blob/main/Parcial1_TAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parcial 1 Teoría de aprendizaje de máquina
Preguntas 2 y 3:

Nicolás Castaño Pérez /1054398549

Mayo 2025


In [ ]:
!pip install scikit-optimize
!pip install streamlit -q #instalación de librerías
!pip install pyngrok
!pip install optuna
!pip install streamlit pandas matplotlib seaborn scikit-learn pyngrok kagglehub
!pip install pyngrok streamlit --quiet

In [ ]:
%%writefile TAMExam1.py
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import time
import altair as alt
import pydeck as pdk

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, SGDRegressor, BayesianRidge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from scipy.stats import loguniform, uniform, randint
from joblib import dump
from urllib.error import URLError

# Configuración de la página de Streamlit
st.set_page_config(layout="wide", page_title="Análisis y Modelización de Ames Housing")

# Título
st.title('Análisis y Modelización de Precios de Viviendas - Ames Housing')

# Documentación Formal: Introducción
st.header('Documentación Formal: Análisis Exploratorio y Modelización')
st.markdown("""
### Introducción
Este dashboard presenta el análisis exploratorio de datos (EDA), el preprocesamiento y la modelización del dataset Ames Housing para predecir el precio de venta ('SalePrice'). El dataset incluye 2930 observaciones y 81 características. Este trabajo cumple con los requisitos del punto 3 del examen de Teoría de Aprendizaje de Máquina.

### Descripción de los Datos
El dataset Ames Housing contiene variables numéricas (como 'GrLivArea', 'TotalBsmtSF') y categóricas (como 'Neighborhood', 'MSZoning'). La variable objetivo es 'SalePrice'.

### Metodología
- **Preprocesamiento:** Manejo de valores faltantes (mediana para numéricas, moda o 'missing' para categóricas), codificación de variables categóricas (One-Hot para nominales, Ordinal para ordinales), ingeniería de características (creación de 'Age' y 'TotalSF'), y escalado de variables numéricas.
- **EDA:** Visualizaciones para entender la distribución de variables y su relación con 'SalePrice'.
- **Modelización:** Entrenamiento de múltiples modelos de regresión con optimización de hiperparámetros (GridSearchCV, RandomizedSearchCV, BayesSearchCV), evaluación con métricas MAE, MSE, R2 y MAPE.
- **Dashboard:** Visualización interactiva de los datos y comparación de los tres mejores modelos.
""")

# Cargar los datos
@st.cache_data
def load_data():
    try:
        df = pd.read_csv('/content/drive/Shareddrives/UNAL_Colab/Teoría de Aprendizaje de Máquina/AmesHousing.csv')
        df.columns = df.columns.str.replace(' ', '_').str.replace('[^A-Za-z0-9_]+', '', regex=True)
        return df
    except FileNotFoundError:
        st.error("Error: Asegúrese de que el archivo 'AmesHousing.csv' esté disponible.")
        return pd.DataFrame()

df = load_data()
if df.empty:
    st.stop()

st.subheader("Datos Originales (primeras 5 filas)")
st.dataframe(df.head())

# Documentación Formal: Preprocesamiento
st.header('Preprocesamiento de Datos')
st.markdown("""
### Metodología de Preprocesamiento
El preprocesamiento asegura que los datos estén limpios y listos para el modelado, abordando los siguientes pasos:

1. **Manejo de Valores Faltantes**
   - **Numéricas:** Se imputaron con la mediana para columnas como 'LotFrontage' y 'GarageYrBlt', ya que es robusta frente a outliers.
   - **Categóricas:** Se imputaron con 'missing' para columnas como 'Alley', 'Fence', 'MiscFeature', 'PoolQC' y 'FireplaceQu', reconociendo que la ausencia de datos puede ser informativa (por ejemplo, 'missing' en 'Alley' indica que no hay callejón). Otras categóricas, como 'MasVnrType' y 'Electrical', se imputaron con la moda para preservar la distribución.
   - **Justificación:** La mediana minimiza el impacto de valores extremos en numéricas, mientras que 'missing' y la moda son adecuadas para categóricas según el contexto.

2. **Codificación de Variables Categóricas**
   - **Nominales:** Columnas como 'MSZoning', 'Neighborhood', 'MasVnrType' se codificaron con One-Hot Encoding, generando variables binarias para cada categoría, eliminando la primera categoría para evitar multicolinealidad.
   - **Ordinales:** Columnas como 'ExterQual', 'BsmtQual', 'HeatingQC' se codificaron con Ordinal Encoding, asignando valores numéricos según un orden definido (por ejemplo, 'Po' < 'Fa' < 'TA' < 'Gd' < 'Ex').
   - **Justificación:** One-Hot Encoding es ideal para variables sin orden inherente, mientras que Ordinal Encoding respeta la jerarquía en variables de calidad o condición.

3. **Ingeniería de Características**
   - **TotalSF:** Suma de 'TotalBsmtSF', '1stFlrSF' y '2ndFlrSF', representando el área total habitable.
   - **Age:** Diferencia entre 'YrSold' y 'YearBuilt', indicando la antigüedad de la propiedad.
   - **Justificación:** Estas características combinan información relevante, reduciendo la dimensionalidad y capturando factores clave que afectan el precio de venta.

4. **Escalado de Variables Numéricas**
   - Se aplicó `StandardScaler` a todas las variables numéricas, transformándolas a una distribución con media 0 y desviación estándar 1.
   - **Justificación:** El escalado es esencial para modelos sensibles a la escala, como KernelRidge o SVR, y mejora la comparabilidad entre características.

5. **Análisis de Correlaciones**
   - Se identificaron pares de características con correlaciones altas (>0.8), como 'TotalBsmtSF' y '1stFlrSF', para evaluar redundancias.
   - **Justificación:** Eliminar características redundantes reduce el riesgo de multicolinealidad y mejora la eficiencia del modelado.

**Guardado de Datos Preprocesados**
Los datos preprocesados se guardan en 'preprocessed_data.csv' para su uso en el entrenamiento de modelos, asegurando consistencia y evitando reprocesamiento redundante.
""")

# Preprocesamiento
@st.cache_data
def preprocess_data(df):
    df_processed = df.copy()
    threshold = len(df_processed) * 0.5
    cols_to_drop_nulls = df_processed.columns[df_processed.isnull().sum() > threshold].tolist()
    df_processed = df_processed.drop(columns=cols_to_drop_nulls)
    st.sidebar.write(f"Columnas eliminadas por exceso de nulos (>50%): {cols_to_drop_nulls}")

    numerical_cols = df_processed.select_dtypes(include=np.number).columns.tolist()
    categorical_cols = df_processed.select_dtypes(include='object').columns.tolist()

    numeric_cols_median = ['LotFrontage', 'GarageYrBlt']
    categorical_cols_mode = ['MasVnrType', 'Electrical']
    categorical_cols_missing = ['Alley', 'Fence', 'MiscFeature', 'PoolQC', 'FireplaceQu']

    for col in numeric_cols_median:
        if col in df_processed.columns and df_processed[col].isnull().any():
            imputer_median = SimpleImputer(strategy='median')
            df_processed[col] = imputer_median.fit_transform(df_processed[[col]]).ravel()

    for col in categorical_cols_mode:
        if col in df_processed.columns and df_processed[col].isnull().any():
            imputer_mode = SimpleImputer(strategy='most_frequent')
            df_processed[col] = imputer_mode.fit_transform(df_processed[[col]]).ravel()

    for col in categorical_cols_missing:
        if col in df_processed.columns and df_processed[col].isnull().any():
            imputer_missing = SimpleImputer(strategy='constant', fill_value='missing')
            df_processed[col] = imputer_missing.fit_transform(df_processed[[col]]).ravel()

    if 'YearBuilt' in df_processed.columns:
        df_processed['Age'] = 2023 - df_processed['YearBuilt']
        df_processed = df_processed.drop(columns=['YearBuilt'])
    if 'YearRemodAdd' in df_processed.columns:
        df_processed['YearsSinceRemodel'] = 2023 - df_processed['YearRemodAdd']
        df_processed = df_processed.drop(columns=['YearRemodAdd'])
    if all(col in df_processed.columns for col in ['TotalBsmtSF', '1stFlrSF', '2ndFlrSF']):
        df_processed['TotalSF'] = df_processed['TotalBsmtSF'] + df_processed['1stFlrSF'] + df_processed['2ndFlrSF']

    nominal_cols = ['MSZoning', 'Neighborhood', 'MasVnrType', 'Exterior1st', 'Exterior2nd', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'SaleType', 'SaleCondition']
    nominal_cols = [col for col in nominal_cols if col in df_processed.columns]
    ordinal_mapping = {
        'ExterQual': ['Po', 'Fa', 'TA', 'Gd', 'Ex', 'missing'],
        'ExterCond': ['Po', 'Fa', 'TA', 'Gd', 'Ex', 'missing'],
        'BsmtQual': ['missing', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
        'BsmtCond': ['missing', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
        'HeatingQC': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
        'KitchenQual': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
        'FireplaceQu': ['missing', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
        'GarageQual': ['missing', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
        'GarageCond': ['missing', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
        'PoolQC': ['missing', 'Fa', 'Gd', 'Ex']
    }
    ordinal_cols = [col for col in ordinal_mapping.keys() if col in df_processed.columns]

    if nominal_cols:
        df_processed = pd.get_dummies(df_processed, columns=nominal_cols, drop_first=True)
    if ordinal_cols:
        ordinal_encoder = OrdinalEncoder(categories=[ordinal_mapping[col] for col in ordinal_cols])
        df_processed[ordinal_cols] = ordinal_encoder.fit_transform(df_processed[ordinal_cols])

    numeric_cols_to_scale = df_processed.select_dtypes(include=np.number).columns.tolist()
    if 'SalePrice' in numeric_cols_to_scale:
        numeric_cols_to_scale.remove('SalePrice')
    if 'Id' in numeric_cols_to_scale:
        numeric_cols_to_scale.remove('Id')
    if numeric_cols_to_scale:
        scaler = StandardScaler()
        df_processed[numeric_cols_to_scale] = scaler.fit_transform(df_processed[numeric_cols_to_scale])

    numeric_df = df_processed.select_dtypes(include=np.number)
    correlation_matrix = numeric_df.corr()
    correlation_threshold = 0.8
    upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
    to_drop_high_corr = [column for column in upper.columns if any(upper[column].abs() > correlation_threshold)]
    st.sidebar.write(f"Columnas con alta correlación (> {correlation_threshold}): {to_drop_high_corr}")

    # Guardar datos preprocesados
    df_processed.to_csv('preprocessed_data.csv', index=False)
    return df_processed

df_processed = preprocess_data(df.copy())
st.subheader("Datos Preprocesados (primeras 5 filas)")
st.dataframe(df_processed.head())

# Documentación Formal: Visualizaciones
st.header('Análisis Exploratorio de Datos')
st.markdown("""
### Visualizaciones Clave
El análisis exploratorio utiliza visualizaciones para identificar patrones, distribuciones y relaciones en el dataset:

1. **Distribución de SalePrice**
   - **Histograma:** Muestra la distribución de los precios de venta, con un sesgo positivo que indica una mayoría de propiedades de menor valor y algunos outliers de alto valor.
   - **Boxplot:** Identifica outliers en los precios altos, sugiriendo la necesidad de transformaciones o manejo de valores extremos.

2. **Distribución de Variables Numéricas**
   - Histogramas y boxplots para variables como 'GrLivArea', 'TotalBsmtSF', 'Age', 'TotalSF' muestran sus distribuciones y posibles outliers, proporcionando información sobre su impacto en el precio.

3. **Relación con SalePrice**
   - Gráficos de dispersión entre 'SalePrice' y las variables más correlacionadas (como 'OverallQual', 'GrLivArea') revelan relaciones lineales y no lineales, destacando características predictivas clave.

4. **Distribución de Variables Categóricas**
   - Gráficos de barras para variables como 'Neighborhood' y 'MSZoning' muestran la frecuencia de categorías, ayudando a entender su distribución y posible influencia en 'SalePrice'.

5. **Matriz de Correlación**
   - Un mapa de calor identifica correlaciones entre variables numéricas, destacando relaciones fuertes y posibles redundancias.
""")

# Visualizaciones
st.sidebar.subheader("Opciones de Visualización")
viz_option = st.sidebar.selectbox("Selecciona una visualización:",
                                  ["Distribución de SalePrice",
                                   "Mapa de Calor de Correlación",
                                   "Relación con SalePrice (Scatter plots)",
                                   "Distribución de Variables Categóricas",
                                   "Distribución de Variables Numéricas"])

if viz_option == "Distribución de SalePrice":
    st.subheader("Distribución de SalePrice")
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    sns.histplot(df['SalePrice'], kde=True, ax=axes[0])
    axes[0].set_title('Distribución de SalePrice')
    axes[0].set_xlabel('SalePrice')
    axes[0].set_ylabel('Frecuencia')
    sns.boxplot(y=df['SalePrice'], ax=axes[1])
    axes[1].set_title('Boxplot de SalePrice')
    axes[1].set_ylabel('SalePrice')
    plt.tight_layout()
    st.pyplot(fig)
    st.markdown("""
    **Análisis:** La distribución de 'SalePrice' muestra un sesgo positivo (skewness > 0.5), con outliers en valores altos. Esto sugiere considerar una transformación logarítmica para normalizar los datos en el modelado.
    """)

elif viz_option == "Mapa de Calor de Correlación":
    st.subheader("Mapa de Calor de Correlación")
    numeric_df = df_processed.select_dtypes(include=np.number)
    if 'SalePrice' in numeric_df.columns:
        correlation_with_saleprice = numeric_df.corr()['SalePrice'].sort_values(ascending=False)
        st.write("Correlación de variables numéricas con SalePrice:")
        st.dataframe(correlation_with_saleprice)
        fig, ax = plt.subplots(figsize=(8, 10))
        sns.heatmap(correlation_with_saleprice.to_frame(), cmap='coolwarm', annot=True, fmt=".2f", cbar=False, ax=ax)
        ax.set_title('Correlación con SalePrice')
        ax.tick_params(axis='y', rotation=0)
        st.pyplot(fig)
        st.markdown("""
        **Análisis:** Variables como 'OverallQual' y 'GrLivArea' tienen correlaciones altas con 'SalePrice', indicando su importancia predictiva. Las correlaciones negativas son menos comunes, pero pueden reflejar factores como la antigüedad ('Age').
        """)

elif viz_option == "Relación con SalePrice (Scatter plots)":
    st.subheader("Relación entre SalePrice y Variables Numéricas Clave")
    numeric_df = df_processed.select_dtypes(include=np.number)
    if 'SalePrice' in numeric_df.columns:
        correlation_with_saleprice = numeric_df.corr()['SalePrice'].sort_values(ascending=False)
        top_n = st.slider("Número de variables más correlacionadas a mostrar:", 3, 10, 5)
        top_correlated_cols = correlation_with_saleprice.head(top_n + 1).index.tolist()
        if 'SalePrice' in top_correlated_cols:
            top_correlated_cols.remove('SalePrice')
        st.write(f"Mostrando la relación de SalePrice con las {len(top_correlated_cols)} variables más correlacionadas:")
        num_cols_plot = 3
        num_rows_plot = int(np.ceil(len(top_correlated_cols) / num_cols_plot))
        fig, axes = plt.subplots(num_rows_plot, num_cols_plot, figsize=(15, 5 * num_rows_plot))
        axes = axes.flatten() if num_rows_plot > 1 else [axes]
        for i, col in enumerate(top_correlated_cols):
            if col in numeric_df.columns:
                sns.scatterplot(x=numeric_df[col], y=numeric_df['SalePrice'], ax=axes[i])
                axes[i].set_title(f'SalePrice vs {col}')
                axes[i].set_xlabel(col)
                axes[i].set_ylabel('SalePrice')
            else:
                axes[i].set_visible(False)
        for j in range(i + 1, len(axes)):
            axes[j].set_visible(False)
        plt.tight_layout()
        st.pyplot(fig)
        st.markdown("""
        **Análisis:** Los gráficos de dispersión muestran relaciones lineales y no lineales entre 'SalePrice' y variables clave, como 'GrLivArea' (positiva) y 'Age' (potencialmente negativa), destacando su relevancia para la predicción.
        """)

elif viz_option == "Distribución de Variables Categóricas":
    st.subheader("Distribución de Variables Categóricas")
    df_original = load_data()
    df_original.columns = df_original.columns.str.replace(' ', '_').str.replace('[^A-Za-z0-9_]+', '', regex=True)
    categorical_cols = df_original.select_dtypes(include='object').columns.tolist()
    for col in categorical_cols:
        if df_original[col].isnull().any():
            df_original[col].fillna('Missing', inplace=True)
    if categorical_cols:
        selected_cols = st.multiselect("Selecciona variables categóricas para visualizar:", categorical_cols, default=categorical_cols[:5])
        if selected_cols:
            num_cols_plot = 2
            num_rows_plot = int(np.ceil(len(selected_cols) / num_cols_plot))
            fig, axes = plt.subplots(num_rows_plot, num_cols_plot, figsize=(15, 5 * num_rows_plot))
            axes = axes.flatten() if num_rows_plot > 1 else [axes]
            for i, col in enumerate(selected_cols):
                if col in df_original.columns and df_original[col].dtype == 'object':
                    sns.countplot(y=col, data=df_original, order=df_original[col].value_counts().index, ax=axes[i])
                    axes[i].set_title(f'Distribución de {col}')
                    axes[i].set_xlabel('Frecuencia')
                    axes[i].set_ylabel(col)
                else:
                    axes[i].set_visible(False)
            for j in range(i + 1, len(axes)):
                axes[j].set_visible(False)
            plt.tight_layout()
            st.pyplot(fig)
            st.markdown("""
            **Análisis:** Los gráficos de barras muestran la distribución de categorías, como 'Neighborhood', revelando la composición del dataset y posibles influencias en 'SalePrice'.
            """)

elif viz_option == "Distribución de Variables Numéricas":
    st.subheader("Distribución de Variables Numéricas")
    numeric_cols = df_processed.select_dtypes(include=np.number).columns.tolist()
    if 'SalePrice' in numeric_cols:
        numeric_cols.remove('SalePrice')
    if numeric_cols:
        selected_cols = st.multiselect("Selecciona variables numéricas para visualizar:", numeric_cols, default=numeric_cols[:5])
        if selected_cols:
            num_cols_plot = 3
            num_rows_plot = int(np.ceil(len(selected_cols) / num_cols_plot)) * 2
            fig, axes = plt.subplots(num_rows_plot, num_cols_plot, figsize=(15, 5 * num_rows_plot/2))
            axes = axes.flatten() if num_rows_plot > 1 else [axes]
            plot_index = 0
            for i, col in enumerate(selected_cols):
                if col in df_processed.columns and df_processed[col].dtype in [np.number]:
                    sns.histplot(df_processed[col], kde=True, ax=axes[plot_index])
                    axes[plot_index].set_title(f'Distribución de {col}')
                    axes[plot_index].set_xlabel(col)
                    axes[plot_index].set_ylabel('Frecuencia')
                    plot_index += 1
                    sns.boxplot(y=df_processed[col], ax=axes[plot_index])
                    axes[plot_index].set_title(f'Boxplot de {col}')
                    axes[plot_index].set_ylabel(col)
                    plot_index += 1
                else:
                    axes[plot_index].set_visible(False)
                    axes[plot_index+1].set_visible(False)
                    plot_index += 2
            for j in range(plot_index, len(axes)):
                axes[j].set_visible(False)
            plt.tight_layout()
            st.pyplot(fig)
            st.markdown("""
            **Análisis:** Los histogramas y boxplots muestran la distribución y presencia de outliers en variables numéricas, como 'GrLivArea' y 'TotalSF', proporcionando información sobre su impacto en el modelado.
            """)

# Documentación Formal: Modelización
# Documentación Formal: Modelización
st.header('Modelización de Datos')
st.markdown("""
### Metodología de Modelización
Se entrenaron y evaluaron múltiples modelos de regresión para predecir 'SalePrice', utilizando validación cruzada de 5 pliegues y optimización de hiperparámetros con GridSearchCV, RandomizedSearchCV y BayesSearchCV. Los modelos son:

- **LinearRegression**: Modelo base para relaciones lineales simples, sin optimización de hiperparámetros debido a su simplicidad.
- **Lasso**: Regularización L1 para selección de características, útil para datasets con multicolinealidad.
- **ElasticNet**: Combina regularización L1 y L2, balanceando selección y estabilidad.
- **KernelRidge**: Captura relaciones no lineales mediante un kernel RBF, adecuado para patrones complejos.
- **SGDRegressor**: Optimización basada en gradiente descendente, eficiente para datasets grandes.
- **BayesianRidge**: Modelo bayesiano robusto a multicolinealidad, con parámetros predeterminados.
- **RandomForestRegressor**: Ensamblado de árboles, robusto a valores faltantes y relaciones no lineales.
- **SVR**: Máquinas de soporte vectorial con kernel RBF, efectivas para relaciones no lineales.

### Optimización de Hiperparámetros
Se utilizaron tres métodos de optimización:
- **GridSearchCV**: Explora exhaustivamente un conjunto finito de hiperparámetros, garantizando la mejor combinación dentro del rango definido.
- **RandomizedSearchCV**: Muestrea aleatoriamente combinaciones de hiperparámetros, eficiente para espacios grandes.
- **BayesSearchCV**: Usa optimización bayesiana para explorar el espacio de hiperparámetros de manera eficiente, priorizando combinaciones prometedoras.

### Rangos de Hiperparámetros y Justificación
- **Lasso**:
  - **GridSearchCV**: `alpha = [0.001, 0.01, 0.1, 1, 10]`. Rango logarítmico para cubrir diferentes niveles de regularización, desde baja (0.001) hasta alta (10), permitiendo selección de características efectiva.
  - **RandomizedSearchCV/BayesSearchCV**: `alpha = loguniform(0.001, 10)`. La distribución loguniforme explora un rango continuo, capturando valores pequeños y grandes para adaptarse a la escala de los datos.
- **ElasticNet**:
  - **GridSearchCV**: `alpha = [0.001, 0.01, 0.1, 1]`, `l1_ratio = [0.1, 0.3, 0.5]`. Rangos discretos para balancear regularización L1 y L2, enfocándose en valores bajos para evitar sobre-regularización.
  - **RandomizedSearchCV/BayesSearchCV**: `alpha = loguniform(0.001, 1)`, `l1_ratio = uniform(0, 1)`. La distribución loguniforme para `alpha` cubre un espectro amplio, mientras que `l1_ratio` explora todo el rango de balance entre L1 y L2.
- **KernelRidge**:
  - **GridSearchCV**: `alpha = [0.001, 0.01, 0.1]`, `gamma = [0.001, 0.01]`. Rangos discretos para regularización y escala del kernel RBF, enfocados en valores pequeños para capturar no linealidades.
  - **RandomizedSearchCV/BayesSearchCV**: `alpha = loguniform(0.001, 1)`, `gamma = loguniform(0.001, 1)`. Distribuciones loguniformes para explorar un rango continuo, adecuado para la sensibilidad del kernel RBF.
- **SGDRegressor**:
  - **GridSearchCV**: `alpha = [0.0001, 0.001]`. Rangos pequeños para regularización, ya que valores grandes pueden causar inestabilidad en el gradiente descendente.
  - **RandomizedSearchCV/BayesSearchCV**: `alpha = loguniform(0.0001, 0.01)`. Rango logarítmico para explorar regularización fina, manteniendo estabilidad.
- **RandomForestRegressor**:
  - **GridSearchCV**: `n_estimators = [50, 100]`, `max_depth = [None, 10]`, `min_samples_split = [2, 5]`. Rangos reducidos para acelerar la optimización, cubriendo un número moderado de árboles y profundidades.
  - **RandomizedSearchCV**: `n_estimators = randint(50, 150)`, `max_depth = [None] + list(range(5, 15))`, `min_samples_split = randint(2, 10)`. Rangos discretos para explorar complejidad del modelo.
  - **BayesSearchCV**: `n_estimators = Integer(50, 150)`, `max_depth = Integer(5, 15)`, `min_samples_split = Integer(2, 10)`. Rangos continuos para una búsqueda más precisa.
- **SVR**:
  - **GridSearchCV**: `C = [1, 10, 100]`, `epsilon = [0.01, 0.1, 0.5]`, `gamma = ['scale', 'auto', 0.001]`. Rangos discretos para regularización, margen y escala del kernel.
  - **RandomizedSearchCV/BayesSearchCV**: `C = loguniform(0.1, 1000)`, `epsilon = uniform(0.001, 1)`, `gamma = loguniform(0.0001, 1)`. Distribuciones continuas para explorar un espacio amplio.
- **LinearRegression y BayesianRidge**: Sin optimización de hiperparámetros, ya que sus parámetros predeterminados son robustos para el dataset.

### Número de Iteraciones
- **RandomizedSearchCV y BayesSearchCV**: Se usaron 10 iteraciones para RandomForestRegressor y 50-100 para otros modelos, para balancear precisión y tiempo de cómputo. Menos iteraciones (10) se usaron en RandomForestRegressor debido a su alto costo computacional, mientras que 50-100 iteraciones en modelos más rápidos (como Lasso, ElasticNet) permiten una exploración más exhaustiva.
- **Justificación**: El número de iteraciones refleja un compromiso entre explorar el espacio de hiperparámetros y mantener la eficiencia computacional, considerando que el dataset Ames Housing (~1460 filas tras división) no requiere búsquedas extensas para modelos lineales, pero sí para modelos no lineales.

### Evaluación
Se utilizó validación cruzada de 5 pliegues para garantizar estimaciones robustas de las métricas MAE, MSE, R2 y MAPE. Los resultados se guardan en un archivo pickle ('/content/model_results.pkl') para su uso en el dashboard de Streamlit. Los mejores estimadores se guardan con joblib para pruebas posteriores.
""")

# Entrenamiento de Modelos
st.subheader("Entrenamiento de Modelos")

# Verificar si los resultados ya existen para evitar reentrenamiento
if not os.path.exists('/content/model_results.pkl'):
    try:
        # Cargar datos preprocesados
        st.write("Cargando datos preprocesados desde preprocessed_data.csv...")
        df = pd.read_csv('preprocessed_data.csv')

        # Separar características y variable objetivo
        st.write("Separando características y variable objetivo...")
        if 'SalePrice' in df.columns:
            y = df['SalePrice']
            X = df.drop('SalePrice', axis=1)
        else:
            st.error("La columna 'SalePrice' no se encontró.")
            st.stop()

        # Asegurarse de que X sea numérico
        X = X.select_dtypes(include=np.number)
        X = X.dropna()
        y = y.loc[X.index]

        # División en conjuntos de entrenamiento y prueba
        st.write("Dividiendo datos en entrenamiento y prueba...")
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Definir métricas
        def mean_absolute_percentage_error(y_true, y_pred):
            y_true, y_pred = np.array(y_true), np.array(y_pred)
            epsilon = 1e-8
            return np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

        scoring = {
            'mae': make_scorer(mean_absolute_error, greater_is_better=False),
            'mse': make_scorer(mean_squared_error, greater_is_better=False),
            'r2': make_scorer(r2_score, greater_is_better=True),
            'mape': make_scorer(mean_absolute_percentage_error, greater_is_better=False)
        }
        scoring_optimizer = {'mae': make_scorer(mean_absolute_error, greater_is_better=False)}

        # Configurar KFold
        kf = KFold(n_splits=5, shuffle=True, random_state=42)

        # Definir modelos y espacios de búsqueda
        models = {
            'LinearRegression': LinearRegression(),
            'Lasso': Lasso(random_state=42, max_iter=10000),
            'ElasticNet': ElasticNet(random_state=42, max_iter=10000),
            'KernelRidge': KernelRidge(kernel='rbf'),
            'SGDRegressor': SGDRegressor(learning_rate='invscaling', early_stopping=True, random_state=42, max_iter=10000),
            'BayesianRidge': BayesianRidge(),
            'RandomForestRegressor': RandomForestRegressor(random_state=42, n_jobs=-1),
            'SVR': SVR(kernel='rbf')
        }

        param_grids = {
            'Lasso': {'alpha': [0.001, 0.01, 0.1, 1, 10]},
            'ElasticNet': {'alpha': [0.001, 0.01, 0.1, 1], 'l1_ratio': [0.1, 0.3, 0.5]},
            'KernelRidge': {'alpha': [0.001, 0.01, 0.1], 'gamma': [0.001, 0.01]},
            'SGDRegressor': {'alpha': [0.0001, 0.001]},
            'RandomForestRegressor': {'n_estimators': [50, 100], 'max_depth': [None, 10], 'min_samples_split': [2, 5]},
            'SVR': {'C': [1, 10, 100], 'epsilon': [0.01, 0.1, 0.5], 'gamma': ['scale', 'auto', 0.001]}
        }

        param_dists = {
            'Lasso': {'alpha': loguniform(0.001, 10)},
            'ElasticNet': {'alpha': loguniform(0.001, 1), 'l1_ratio': uniform(0, 1)},
            'KernelRidge': {'alpha': loguniform(0.001, 1), 'gamma': loguniform(0.001, 1)},
            'SGDRegressor': {'alpha': loguniform(0.0001, 0.01)},
            'RandomForestRegressor': {'n_estimators': randint(50, 150), 'max_depth': [None] + list(range(5, 15)), 'min_samples_split': randint(2, 10)},
            'SVR': {'C': loguniform(0.1, 1000), 'epsilon': uniform(0.001, 1), 'gamma': loguniform(0.0001, 1)}
        }

        param_spaces = {
            'Lasso': {'alpha': Real(0.001, 10, prior='log-uniform')},
            'ElasticNet': {'alpha': Real(0.001, 1, prior='log-uniform'), 'l1_ratio': Real(0, 1, prior='uniform')},
            'KernelRidge': {'alpha': Real(0.001, 1, prior='log-uniform'), 'gamma': Real(0.001, 1, prior='log-uniform')},
            'SGDRegressor': {'alpha': Real(0.0001, 0.01, prior='log-uniform')},
            'RandomForestRegressor': {'n_estimators': Integer(50, 150), 'max_depth': Integer(5, 15), 'min_samples_split': Integer(2, 10)},
            'SVR': {'C': Real(0.1, 1000, prior='log-uniform'), 'epsilon': Real(0.001, 1, prior='uniform'), 'gamma': Real(0.0001, 1, prior='log-uniform')}
        }

        # Entrenar y evaluar modelos
        st.write("Iniciando entrenamiento de modelos...")
        model_results = {}
        for name, model in models.items():
            st.write(f"Entrenando modelo: {name}")
            start_time = time.time()

            # GridSearchCV
            if name not in ['LinearRegression', 'BayesianRidge']:
                st.write(f"Optimizando {name} con GridSearchCV...")
                grid_search = GridSearchCV(estimator=model, param_grid=param_grids[name], scoring=scoring_optimizer, refit='mae', cv=kf, verbose=1, n_jobs=-1)
                grid_search.fit(X_train, y_train)
                grid_results = cross_validate(grid_search.best_estimator_, X_train, y_train, cv=kf, scoring=scoring)
                model_results[f"{name}_Grid"] = {
                    'MAE': -grid_results['test_mae'].mean(),
                    'MSE': -grid_results['test_mse'].mean(),
                    'R2': grid_results['test_r2'].mean(),
                    'MAPE': -grid_results['test_mape'].mean(),
                    'Time (s)': time.time() - start_time,
                    'Best Estimator': grid_search.best_estimator_,
                    'Best Params': grid_search.best_params_
                }

            # RandomizedSearchCV
            if name not in ['LinearRegression', 'BayesianRidge']:
                st.write(f"Optimizando {name} con RandomizedSearchCV...")
                random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dists[name], n_iter=50 if name != 'RandomForestRegressor' else 10, scoring=scoring_optimizer, refit='mae', cv=kf, verbose=1, random_state=42, n_jobs=-1)
                random_search.fit(X_train, y_train)
                random_results = cross_validate(random_search.best_estimator_, X_train, y_train, cv=kf, scoring=scoring)
                model_results[f"{name}_Random"] = {
                    'MAE': -random_results['test_mae'].mean(),
                    'MSE': -random_results['test_mse'].mean(),
                    'R2': random_results['test_r2'].mean(),
                    'MAPE': -random_results['test_mape'].mean(),
                    'Time (s)': time.time() - start_time,
                    'Best Estimator': random_search.best_estimator_,
                    'Best Params': random_search.best_params_
                }

            # BayesSearchCV
            if name not in ['LinearRegression', 'BayesianRidge']:
                st.write(f"Optimizando {name} con BayesSearchCV...")
                bayes_search = BayesSearchCV(estimator=model, search_spaces=param_spaces[name], n_iter=50 if name != 'RandomForestRegressor' else 10, scoring=scoring_optimizer, refit='mae', cv=kf, verbose=1, random_state=42, n_jobs=-1)
                bayes_search.fit(X_train, y_train)
                bayes_results = cross_validate(bayes_search.best_estimator_, X_train, y_train, cv=kf, scoring=scoring)
                model_results[f"{name}_Bayes"] = {
                    'MAE': -bayes_results['test_mae'].mean(),
                    'MSE': -bayes_results['test_mse'].mean(),
                    'R2': bayes_results['test_r2'].mean(),
                    'MAPE': -bayes_results['test_mape'].mean(),
                    'Time (s)': time.time() - start_time,
                    'Best Estimator': bayes_search.best_estimator_,
                    'Best Params': bayes_search.best_params_
                }

            # Si no requiere optimización, evaluar directamente
            if name in ['LinearRegression', 'BayesianRidge']:
                st.write(f"Evaluando {name} sin optimización...")
                model.fit(X_train, y_train)
                results = cross_validate(model, X_train, y_train, cv=kf, scoring=scoring)
                model_results[name] = {
                    'MAE': -results['test_mae'].mean(),
                    'MSE': -results['test_mse'].mean(),
                    'R2': results['test_r2'].mean(),
                    'MAPE': -results['test_mape'].mean(),
                    'Time (s)': time.time() - start_time,
                    'Best Estimator': model,
                    'Best Params': 'Default'
                }

        # Guardar resultados en archivo pickle
        st.write("Guardando resultados en /content/model_results.pkl...")
        with open('/content/model_results.pkl', 'wb') as f:
            pickle.dump(model_results, f)

        # Guardar los mejores estimadores con joblib
        st.write("Guardando los mejores estimadores con joblib...")
        for key, result in model_results.items():
            if '_Grid' in key or '_Random' in key or '_Bayes' in key:
                model_name = key.split('_')[0]
                dump(result['Best Estimator'], f"{model_name}_best_model.joblib")
            elif key in ['LinearRegression', 'BayesianRidge']:
                dump(result['Best Estimator'], f"{key}_model.joblib")

        st.write("Entrenamiento y evaluación completados con éxito.")

    except Exception as e:
        st.error(f"Error durante la ejecución: {e}")
        st.stop()

# Visualización de Resultados del Modelado
st.header("Resultados del Entrenamiento de Modelos")

# Verificar si el archivo existe
if os.path.exists("/content/model_results.pkl"):
    with open("/content/model_results.pkl", "rb") as f:
        model_results = pickle.load(f)

    # Convertir resultados en DataFrame para visualización
    results_summary = []
    for model_name, metrics in model_results.items():
        results_summary.append({
            "Modelo": model_name,
            "MAE": round(metrics["MAE"], 2),
            "MSE": round(metrics["MSE"], 2),
            "R2": round(metrics["R2"], 3),
            "MAPE (%)": round(metrics["MAPE"], 2),
            "Tiempo (s)": round(metrics["Time (s)"], 2)
        })
    results_df = pd.DataFrame(results_summary)
    st.subheader("Resumen de Métricas por Modelo")
    st.dataframe(results_df.sort_values(by="MAE"))

    # Visualización de Métricas
    metric_to_plot = st.selectbox("Selecciona una métrica para comparar modelos:", ["MAE", "MSE", "R2", "MAPE (%)", "Tiempo (s)"])
    fig, ax = plt.subplots(figsize=(12, 6))
    sns.barplot(data=results_df.sort_values(by=metric_to_plot, ascending=(metric_to_plot != "R2")), x="Modelo", y=metric_to_plot, ax=ax)
    ax.set_title(f"Comparación de Modelos según {metric_to_plot}")
    ax.set_ylabel(metric_to_plot)
    ax.set_xlabel("Modelo")
    plt.xticks(rotation=45)
    st.pyplot(fig)

    # Mostrar los 3 mejores modelos según MAE
    st.subheader("Top 3 Modelos según MAE")
    top_3_models = results_df.sort_values(by="MAE").head(3)
    st.dataframe(top_3_models)

    # Comparación gráfica de los 3 mejores modelos
    st.subheader("Comparación Gráfica de los 3 Mejores Modelos según MAE")
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.barplot(data=top_3_models, x="Modelo", y="MAE", ax=ax)
    ax.set_title("Top 3 Modelos según MAE")
    ax.set_ylabel("MAE")
    ax.set_xlabel("Modelo")
    plt.xticks(rotation=45)
    st.pyplot(fig)

else:
    st.warning("El archivo '/content/model_results.pkl' no se encuentra. Por favor, ejecuta el entrenamiento de modelos primero.")

# Documentación Formal: Conclusión
st.header('Conclusión')
st.markdown("""
El análisis exploratorio, preprocesamiento y modelización del dataset Ames Housing han permitido preparar los datos y entrenar modelos predictivos para 'SalePrice'. El preprocesamiento abordó valores faltantes, codificación de variables categóricas, ingeniería de características y escalado. Las visualizaciones del EDA identificaron patrones clave, como el sesgo en 'SalePrice' y la importancia de variables como 'OverallQual'. Los modelos fueron optimizados y evaluados, con los mejores resultados guardados para análisis futuros. Este dashboard permite explorar los datos y comparar modelos de manera interactiva.
""")

# ... (código previo de TAM.py hasta la sección de "Resultados del Entrenamiento de Modelos")

# Visualización de Resultados del Modelado
st.header("Resultados del Entrenamiento de Modelos")

# Verificar si el archivo existe
if os.path.exists("/content/model_results.pkl"):
    with open("/content/model_results.pkl", "rb") as f:
        model_results = pickle.load(f)

    # Convertir resultados en DataFrame para visualización
    results_summary = []

    # Ensure X_test and y_test are available when running the prediction visualization
if 'X_test' not in locals():
    st.warning("Variables de prueba (X_test, y_test) no encontradas. Volviendo a cargar y dividir datos...")
    try:
        df = pd.read_csv('preprocessed_data.csv')
        if 'SalePrice' in df.columns:
            y = df['SalePrice']
            X = df.drop('SalePrice', axis=1)
        else:

            st.stop()
        X = X.select_dtypes(include=np.number)
        X = X.dropna()
        y = y.loc[X.index]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    except FileNotFoundError:
        st.error("Error: Asegúrese de que el archivo 'preprocessed_data.csv' esté disponible.")
        st.stop()
    for model_name, metrics in model_results.items():
        results_summary.append({
            "Modelo": model_name,
            "MAE": round(metrics["MAE"], 2),
            "MSE": round(metrics["MSE"], 2),
            "R2": round(metrics["R2"], 3),
            "MAPE (%)": round(metrics["MAPE"], 2),
            "Tiempo (s)": round(metrics["Time (s)"], 2)
        })
    results_df = pd.DataFrame(results_summary)
    st.subheader("Resumen de Métricas por Modelo")
    st.dataframe(results_df.sort_values(by="MAE"))

    # Visualización de Métricas
    metric_to_plot = st.selectbox("Selecciona una métrica para comparar modelos:", ["MAE", "MSE", "R2", "MAPE (%)", "Tiempo (s)"], key="metric_selector_1")
    fig, ax = plt.subplots(figsize=(12, 6))
    sns.barplot(data=results_df.sort_values(by=metric_to_plot, ascending=(metric_to_plot != "R2")), x="Modelo", y=metric_to_plot, ax=ax)
    ax.set_title(f"Comparación de Modelos según {metric_to_plot}")
    ax.set_ylabel(metric_to_plot)
    ax.set_xlabel("Modelo")
    plt.xticks(rotation=45)
    st.pyplot(fig)

    # Scatter Plot de Valores Predichos vs Reales para Todos los Modelos
    st.subheader("Scatter Plot de Valores Predichos vs Reales para Todos los Modelos")
    st.markdown("""
    **Descripción:** Este gráfico muestra los valores predichos frente a los valores reales de 'SalePrice'. Los puntos cercanos a la línea diagonal (y=x) indican predicciones precisas.
    """)
    selected_model = st.selectbox("Selecciona un modelo:", list(model_results.keys()))
    best_estimator = model_results[selected_model]['Best Estimator']
    y_pred = best_estimator.predict(X_test)

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.scatter(y_test, y_pred, alpha=0.5, color='blue')
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    ax.set_xlabel('Valores Reales')
    ax.set_ylabel('Valores Predichos')
    ax.set_title(f'Scatter Plot para {selected_model}')
    st.pyplot(fig)

    # Gráfica de Residuos para Todos los Modelos
    st.subheader("Gráfica de Residuos para Todos los Modelos")
    st.markdown("""
    **Descripción:** Este gráfico muestra los residuos (valor real - predicho) frente a los valores predichos. Un patrón aleatorio alrededor de cero indica un modelo robusto.
    """)
    selected_model = st.selectbox("Selecciona un modelo para residuos:", list(model_results.keys()))
    best_estimator = model_results[selected_model]['Best Estimator']
    y_pred = best_estimator.predict(X_test)
    residuals = y_test - y_pred

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.scatter(y_pred, residuals, alpha=0.5, color='green')
    ax.axhline(y=0, color='r', linestyle='--')
    ax.set_xlabel('Valores Predichos')
    ax.set_ylabel('Residuos')
    ax.set_title(f'Residuos para {selected_model}')
    st.pyplot(fig)

    # Gráfica de Importancia de Características para Modelos Aplicables
    st.subheader("Importancia de Características para Modelos Aplicables")
    st.markdown("""
    **Descripción:** Este gráfico muestra la importancia de las características para modelos como RandomForestRegressor. Para otros modelos, se indica si no es aplicable.
    """)
    rf_models = [key for key in model_results.keys() if '_RandomForest' in key]
    if rf_models:
        selected_rf_model = st.selectbox("Selecciona un modelo RandomForest:", rf_models)
        best_estimator = model_results[selected_rf_model]['Best Estimator']
        if hasattr(best_estimator, 'feature_importances_'):
            importances = best_estimator.feature_importances_
            feature_names = X.columns
            importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances}).sort_values('importance', ascending=False).head(10)
            fig, ax = plt.subplots(figsize=(10, 6))
            sns.barplot(x='importance', y='feature', data=importance_df, ax=ax)
            ax.set_title(f'Importancia de Características para {selected_rf_model}')
            ax.set_xlabel('Importancia')
            ax.set_ylabel('Características')
            st.pyplot(fig)
        else:
            st.write(f"{selected_rf_model} no proporciona importancia de características.")
    else:
        st.write("No hay modelos RandomForest disponibles.")

    # Mostrar los 3 mejores modelos según MAE
    st.subheader("Top 3 Modelos según MAE")
    st.markdown("""
    **Descripción:** Esta tabla muestra los tres modelos con el menor MAE, junto con sus métricas de rendimiento.
    """)
    top_3_models = results_df.sort_values(by="MAE").head(3)
    st.dataframe(top_3_models)

    # Comparación Gráfica de los 3 Mejores Modelos
    st.subheader("Comparación Gráfica de los 3 Mejores Modelos según MAE")
    st.markdown("""
    **Descripción:** Este gráfico de barras compara el MAE de los tres mejores modelos, destacando su rendimiento relativo.
    """)
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.barplot(data=top_3_models, x="Modelo", y="MAE", ax=ax)
    ax.set_title("Top 3 Modelos según MAE")
    ax.set_ylabel("MAE")
    ax.set_xlabel("Modelo")
    plt.xticks(rotation=45)
    st.pyplot(fig)

    # Scatter Plot de los 3 Mejores Modelos
    st.subheader("Scatter Plot de los 3 Mejores Modelos según MAE")
    st.markdown("""
    **Descripción:** Estos gráficos muestran los valores predichos frente a los reales para los tres mejores modelos, permitiendo una comparación visual de su precisión.
    """)
    top_3 = sorted(model_results.items(), key=lambda x: x[1]['MAE'])[:3]
    top_3_keys = [key for key, _ in top_3]

    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    for i, key in enumerate(top_3_keys):
        best_estimator = model_results[key]['Best Estimator']
        y_pred = best_estimator.predict(X_test)
        axes[i].scatter(y_test, y_pred, alpha=0.5, color='blue')
        axes[i].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
        axes[i].set_xlabel('Valores Reales')
        axes[i].set_ylabel('Valores Predichos')
        axes[i].set_title(f'{key}')
    plt.tight_layout()
    st.pyplot(fig)

    # Gráfica de Residuos para los 3 Mejores Modelos
    st.subheader("Gráfica de Residuos para los 3 Mejores Modelos según MAE")
    st.markdown("""
    **Descripción:** Estos gráficos muestran los residuos frente a los valores predichos para los tres mejores modelos, ayudando a evaluar la consistencia de los errores.
    """)
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    for i, key in enumerate(top_3_keys):
        best_estimator = model_results[key]['Best Estimator']
        y_pred = best_estimator.predict(X_test)
        residuals = y_test - y_pred
        axes[i].scatter(y_pred, residuals, alpha=0.5, color='green')
        axes[i].axhline(y=0, color='r', linestyle='--')
        axes[i].set_xlabel('Valores Predichos')
        axes[i].set_ylabel('Residuos')
        axes[i].set_title(f'{key}')
    plt.tight_layout()
    st.pyplot(fig)

    # Distribución de Errores para los 3 Mejores Modelos
    st.subheader("Distribución de Errores para los 3 Mejores Modelos según MAE")
    st.markdown("""
    **Descripción:** Estos histogramas muestran la distribución de los residuos para los tres mejores modelos, permitiendo comparar la estabilidad de sus errores.
    """)
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    for i, key in enumerate(top_3_keys):
        best_estimator = model_results[key]['Best Estimator']
        y_pred = best_estimator.predict(X_test)
        residuals = y_test - y_pred
        sns.histplot(residuals, kde=True, ax=axes[i], color='purple')
        axes[i].set_title(f'Distribución de Residuos para {key}')
        axes[i].set_xlabel('Residuos')
    plt.tight_layout()
    st.pyplot(fig)

else:
    st.warning("El archivo '/content/model_results.pkl' no se encuentra. Por favor, ejecuta el entrenamiento de modelos primero.")

# Documentación Formal: Conclusión
st.header('Conclusión')
st.markdown("""
El análisis exploratorio, preprocesamiento y modelización del dataset Ames Housing han permitido preparar los datos y entrenar modelos predictivos para 'SalePrice'. El preprocesamiento abordó valores faltantes, codificación de variables categóricas, ingeniería de características y escalado. Las visualizaciones del EDA identificaron patrones clave, como el sesgo en 'SalePrice' y la importancia de variables como 'OverallQual'. Los modelos fueron optimizados y evaluados, con los mejores resultados guardados para análisis futuros. Este dashboard permite explorar los datos y comparar modelos de manera interactiva, con gráficos que destacan la precisión, los errores y la importancia de las características.
""")


In [ ]:
!df -h

# Dashboard

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

#Ejecutar Streamlit
!streamlit run TAMExam1.py &>/content/logs.txt & #Cambiar TAMExam1.py por el nombre de tu archivo principal

#Exponer el puerto 8501 con Cloudflare Tunnel
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

#Leer la URL pública generada por Cloudflare
import time
time.sleep(5)  # Esperar que se genere la URL

import re
found_context = False  # Indicador para saber si estamos en la sección correcta

with open('/content/cloudflared.log') as f:
    for line in f:
        #Detecta el inicio del contexto que nos interesa
        if "Your quick Tunnel has been created" in line:
            found_context = True

        #Busca una URL si ya se encontró el contexto relevante
        if found_context:
            match = re.search(r'https?://\S+', line)
            if match:
                url = match.group(0)  #Extrae la URL encontrada
                print(f'Tu aplicación está disponible en: {url}')
                break  #Termina el bucle después de encontrar la URL

In [ ]:
import os

res = input("Digite (1) para finalizar la ejecución del Dashboard: ")

if res.upper() == "1":
    os.system("pkill streamlit")  # Termina el proceso de Streamlit
    print("El proceso de Streamlit ha sido finalizado.")

In [ ]:
!kill -9 <PID>